# Read a TIF file and convert it into NetCDF format

In this notebook we first read a TIF file, we flip the data upside down, we create a meshgrid with the latitude and longitude provided by the TIF file and we ultimately store all the data into a NetCDF file. 

We first load useful libraries

In [8]:
import numpy as np
import rasterio
from netCDF4 import Dataset
import warnings
warnings.filterwarnings("ignore")

We specify the data path

In [9]:
dat = rasterio.open(r"../../DATA/chile_esa_cci.tif")
data = dat.read()[0]

We check the content of the data variable, where we obtain a matrix with the dimensions of the latitude and longitude

In [10]:
data

array([[210, 210, 210, ..., 120, 120, 120],
       [210, 210, 210, ..., 120, 120,  30],
       [210, 210, 210, ..., 120,  30,  30],
       ...,
       [210, 210, 210, ..., 120, 100, 100],
       [210, 210, 210, ..., 100, 120, 100],
       [210, 210, 210, ..., 100, 100, 100]], dtype=uint8)

We flip the map upside down to correct default ordering

In [11]:
data = np.flipud(data)  

We now define a mesh grid using the boundaries of the input data and the same spacing as the native latitude and longitude coordinates

In [12]:
boundaries=[dat.bounds.left,dat.bounds.right,dat.bounds.bottom,dat.bounds.top]
boundaries

[-74.06944444444444,
 -71.06666666666666,
 -38.70555555555556,
 -35.70277777777778]

We use the .meshgrid( ) from the Numpy package to create the new grid of coordinates

In [13]:
loni = np.linspace(dat.bounds.left, dat.bounds.right,len(data[0]))
lati = np.linspace(dat.bounds.bottom, dat.bounds.top,len(data))
lons, lats = np.meshgrid(loni,lati)

We build the NetCDF file with the meshgrid we have created and the data into generic var, lat and lon variables. 

In [14]:
ny, nx = (len(lats), len(lons[0]))
lon = loni
lat = lati
dataout = data
output = Dataset('../../OUTPUT/chile_esa_cci.nc','w','NETCDF3'); 
output.createDimension('lon',nx);
output.createDimension('lat',ny);
lonvar = output.createVariable('lon','float32',('lon'));lonvar[:] = lon;
latvar = output.createVariable('lat','float32',('lat'));latvar[:] = lat;
var = output.createVariable('var','float32',('lat','lon'));var.setncattr('units','type');var[:] = dataout;
output.close();